<a href="https://colab.research.google.com/github/IBM-EPBL/IBM-Project-24971-1659951514/blob/master/Assignments/M2_Lead/Assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Problem Statement :- SMS SPAM Classification

Problem Statement:
Over recent years, as the popularity of mobile phone devices has increased, Short
Message Service (SMS) has grown into a multi-billion dollar industry. At the same time,
reduction in the cost of messaging services has resulted in growth in unsolicited
commercial advertisements (spams) being sent to mobile phones. Due to Spam SMS,
Mobile service providers suffer from some sort of financial problems as well as it reduces
calling time for users. Unfortunately, if the user accesses such Spam SMS they may face
the problem of virus or malware. When SMS arrives at mobile it will disturb mobile user
privacy and concentration. It may lead to frustration for the user. So Spam SMS is one of
the major issues in the wireless communication world and it grows day by day.

Loading Dataset

In [10]:
from google.colab import files
data_to_load = files.upload()

Saving spam.csv to spam.csv


In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

Importing NLTK libraries

In [15]:
import csv
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
nltk.download('stopwords')  
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Reading dataset and preprocessing

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
cd/content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [19]:
df = pd.read_csv('/content/drive/MyDrive/content/spam.csv',delimiter=',',encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [20]:
df.drop(['Unnamed: 2','Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True) 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [21]:
df.groupby(['v1']).size()

v1
ham     4825
spam     747
dtype: int64

In [22]:
#Label Encoding Required Column
X = df.v2
Y = df.v1
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [23]:
# Test and train data split 
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [29]:
# Tokenisation function
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = pad_sequences(sequences,maxlen=max_len)

In [30]:
#LSTM model
inputs = Input(name='InputLayer',shape=[max_len])
layer = Embedding(max_words,50,input_length=max_len)(inputs)
layer = LSTM(64)(layer)
layer = Dense(256,name='FullyConnectedLayer1')(layer)
layer = Activation('relu')(layer)
layer = Dropout(0.5)(layer)
layer = Dense(1,name='OutputLayer')(layer)
layer = Activation('sigmoid')(layer)

In [31]:
model = Model(inputs=inputs,outputs=layer)
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 InputLayer (InputLayer)     [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FullyConnectedLayer1 (Dense  (None, 256)              16640     
 )                                                               
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                             

In [32]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=25,validation_split=0.2)

Epoch 1/25
30/30 [==============================] - 11s 287ms/step - loss: 0.3232 - accuracy: 0.8757 - val_loss: 0.1439 - val_accuracy: 0.9546
Epoch 2/25
30/30 [==============================] - 8s 266ms/step - loss: 0.0825 - accuracy: 0.9799 - val_loss: 0.0724 - val_accuracy: 0.9747
Epoch 3/25
30/30 [==============================] - 8s 265ms/step - loss: 0.0456 - accuracy: 0.9884 - val_loss: 0.0938 - val_accuracy: 0.9778
Epoch 4/25
30/30 [==============================] - 10s 343ms/step - loss: 0.0317 - accuracy: 0.9908 - val_loss: 0.0677 - val_accuracy: 0.9789
Epoch 5/25
30/30 [==============================] - 10s 325ms/step - loss: 0.0270 - accuracy: 0.9923 - val_loss: 0.0844 - val_accuracy: 0.9778
Epoch 6/25
30/30 [==============================] - 8s 268ms/step - loss: 0.0205 - accuracy: 0.9942 - val_loss: 0.0976 - val_accuracy: 0.9778
Epoch 7/25
30/30 [==============================] - 8s 266ms/step - loss: 0.0161 - accuracy: 0.9955 - val_loss: 0.1214 - val_accuracy: 0.9789
Epo

In [33]:
model.save("Ai_Spam_Identifier")

In [35]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)

In [36]:
accuracy = model.evaluate(test_sequences_matrix,Y_test)
print('Accuracy: {:0.3f}'.format(accuracy[1]))

27/27 [==============================] - 1s 22ms/step - loss: 0.1368 - accuracy: 0.9821
Accuracy: 0.982


In [37]:
y_pred = model.predict(test_sequences_matrix)
print(y_pred[25:40].round(3))

27/27 [==============================] - 2s 22ms/step
[[0.   ]
 [0.   ]
 [0.104]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [1.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [1.   ]
 [0.   ]
 [1.   ]]


In [38]:
print(Y_test[25:40])

[[0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]]
